In [18]:
import torch
from tensornetworks_pytorch.TNModels import PosMPS, Born
print(torch.__version__)

1.8.0.dev20201128


Use some data

In [19]:
import pickle
with open('datasets/lymphography', 'rb') as f:
        a=pickle.load(f)
X=a[0]
X=X.astype(int)

print("data shape:", X.shape)
print("range of X values:", X.max()+1)

data shape: (148, 19)
range of X values: 8


In [22]:
mps = PosMPS(D=2, d=8, verbose=True)
rBorn = Born(D=2, d=8, dtype=torch.float, verbose=True) 
cBorn = Born(D=2, d=8, dtype=torch.cfloat, verbose=True)
models = (mps, rBorn, cBorn)
for model in models:
    print(model.core.shape, model.name)
print("===")
for model in models:
    model.fit(X, 8)

torch.Size([8, 2, 2]) Positive MPS
torch.Size([8, 2, 2]) Born model torch.float32
torch.Size([8, 2, 2]) Born model torch.complex64
===
logprob tensor(-40.0125, grad_fn=<SubBackward0>)
logprob tensor(-39.7624, grad_fn=<SubBackward0>)
logprob tensor(-33.1358, grad_fn=<SubBackward0>)


In [21]:
model = rBorn
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for ep in range(100):
    print(ep)
    model.zero_grad()
    loss = - model(X[0])
    loss.backward()
    optimizer.step()

0
logprob tensor(-43.3111, dtype=torch.float64, grad_fn=<SubBackward0>)
1
logprob tensor(-21.3335, dtype=torch.float64, grad_fn=<SubBackward0>)
2
logprob tensor(-18.9865, dtype=torch.float64, grad_fn=<SubBackward0>)
3
logprob tensor(-18.2378, dtype=torch.float64, grad_fn=<SubBackward0>)
4
logprob tensor(-17.5347, dtype=torch.float64, grad_fn=<SubBackward0>)
5
logprob tensor(-16.7482, dtype=torch.float64, grad_fn=<SubBackward0>)
6
logprob tensor(-16.1996, dtype=torch.float64, grad_fn=<SubBackward0>)
7
logprob tensor(-16.0232, dtype=torch.float64, grad_fn=<SubBackward0>)
8
logprob tensor(-15.9079, dtype=torch.float64, grad_fn=<SubBackward0>)
9
logprob tensor(-15.8290, dtype=torch.float64, grad_fn=<SubBackward0>)
10
logprob tensor(-15.7724, dtype=torch.float64, grad_fn=<SubBackward0>)
11
logprob tensor(-15.7295, dtype=torch.float64, grad_fn=<SubBackward0>)
12
logprob tensor(-15.6955, dtype=torch.float64, grad_fn=<SubBackward0>)
13
logprob tensor(-15.6671, dtype=torch.float64, grad_fn=<Sub